In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
import numpy as np
import matplotlib.pyplot as plt


from HOTS.Event import Event
from HOTS.Layer import Filter, ClusteringLayer
from HOTS.Monitor import DisplayImage, DisplayHisto
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier



# Defining General Parameter

In [ ]:
SaveCluster = True
LoadCluster = True

ClusterFile = '../Records/ClassifyingNotebook/Clusters.pkl'
tau = 0.02
R = 2
nb_cluster = 4
ImageSize = (32,32)

## Training the Cluster on 5 images

In [4]:
image_list = [1, 32, 19, 22, 29]
event0 = Event(ImageSize=(32,32), ListPolarities=[-1,1])

event0.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=image_list)

InputFilter = Filter(event0)
event1 = InputFilter.FilterNHBD(threshold=15,neighbourhood=2)

L1 = ClusteringLayer(event1,verbose=1)
event2, ClusterLayer1= L1.TrainLayer(tau=tau, R=R, nb_cluster=nb_cluster)
L2 = ClusteringLayer(event2,verbose=1)
event3, ClusterLayer2 = L2.TrainLayer(tau=10*tau, R=2*R, nb_cluster=2*nb_cluster)
L3 = ClusteringLayer(event3,verbose=1)
event4, ClusterLayer3 = L3.TrainLayer(tau=10*10*tau, R= 2*2*R, nb_cluster=2*2*nb_cluster)


if SaveCluster==True:
    SaveObject([ClusterLayer1,ClusterLayer2,ClusterLayer3],ClusterFile)

## Generating Histogram for the training samples

In [5]:
if LoadCluster==True:
    ClusterLayer1, ClausterLayer2, ClusterLayer3 = LoadObject(ClusterFile)

training_sample = list(np.arange(36))
event0_tr = Event(ImageSize=(32,32), ListPolarities=[-1,1])

event0_tr.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=training_sample)

InputFilter = Filter(event0_tr)
event1_tr = InputFilter.FilterNHBD(threshold=15,neighbourhood=2)

L1 = ClusteringLayer(event1_tr,verbose=0)
event2_tr = L1.RunLayer(tau=tau, R=R, Cluster=ClusterLayer1)
L2 = ClusteringLayer(event2_tr,verbose=0)
event3_tr = L2.RunLayer(tau=10*tau, R=2*R, Cluster=ClusterLayer2)
L3 = ClusteringLayer(event3_tr,verbose=0)
event4_tr = L3.RunLayer(tau=10*10*tau, R= 2*2*R, Cluster=ClusterLayer3)

chargement des images [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


## Generating Histograms for the test sample

In [6]:
if LoadCluster==True:
    ClusterLayer1, ClausterLayer2, ClusterLayer3 = LoadObject(ClusterFile)

training_sample = list(np.arange(36,76))
event0_te = Event(ImageSize=(32,32), ListPolarities=[-1,1])

event0_te.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=training_sample)

InputFilter = Filter(event0_te)
event1_te = InputFilter.FilterNHBD(threshold=15,neighbourhood=2)

L1 = ClusteringLayer(event1_te,verbose=0)
event2_te = L1.RunLayer(tau=tau, R=R, Cluster=ClusterLayer1)
L2 = ClusteringLayer(event2_te,verbose=0)
event3_te = L2.RunLayer(tau=10*tau, R=2*R, Cluster=ClusterLayer2)
L3 = ClusteringLayer(event3_te,verbose=0)
event4_te = L3.RunLayer(tau=10*10*tau, R= 2*2*R, Cluster=ClusterLayer3)

chargement des images [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In [8]:
label_list=LoadObject('../Data/alphabet_label.pkl')
#with open( '../Data/alphabet_label.pkl','rb') as file:
#    label_list = pickle.load(file)

In [21]:
Classif = Classifier(event4_tr,event4_te,label_list)
label_eucl,label_norm,label_battacha = Classif.HistogramDistance()

(40, 16)
(36, 16)


NameError: name 'test_label' is not defined

In [26]:
for idx,img in enumerate(range(36,76)):
    if idx!= 0:
        test_label = np.vstack((test_label,label_list[img]))
    else:
        test_label = label_list[img]
        


[('X', 14812), ('F', 11472), ('Y', 10978), ('G', 15578), ('Z', 14688), ('H', 16841), ('0', 14493), ('I', 6346), ('1', 9017), ('J', 8130), ('2', 12730), ('K', 15852), ('3', 12331), ('L', 10718), ('4', 14178), ('M', 28692), ('5', 12688), ('N', 18558), ('6', 14169), ('O', 15664), ('7', 9441), ('P', 13586), ('8', 14715), ('Q', 15812), ('9', 15067), ('R', 16944), ('S', 12674), ('A', 15604), ('T', 9524), ('B', 18270), ('U', 15065), ('C', 12510), ('V', 14331), ('D', 17193), ('W', 27015), ('E', 15122), ('X', 14646), ('F', 11356), ('Y', 10149), ('G', 15511), ('Z', 14019), ('H', 17054), ('0', 14819), ('I', 8167), ('1', 8644), ('J', 8143), ('2', 11447), ('K', 15819), ('3', 11978), ('L', 8303), ('4', 12605), ('M', 28315), ('5', 11910), ('N', 18447), ('6', 13574), ('O', 15313), ('7', 8628), ('P', 13562), ('8', 13507), ('Q', 15990), ('9', 13864), ('R', 16820), ('S', 12001), ('A', 15440), ('T', 9098), ('B', 17803), ('U', 14373), ('C', 12621), ('V', 13523), ('D', 17290), ('W', 27736), ('E', 15213), ('

In [28]:
nb_sample = test_label.shape[0]
accuracy_eu = (np.sum(test_label==label_eucl)/nb_sample)*100
accuracy_norm = (np.sum(test_label==label_norm)/nb_sample)*100
accuracy_battacha = (np.sum(test_label==label_battacha)/nb_sample)*100
#accuracy_KL = (np.sum(test_label==label_KL)/nb_sample)*100
print("Accuracy with euclidian classsifier: {0:.2f} %".format(accuracy_eu))
print("Accuracy with normalized classsifier: {0:.2f} %".format(accuracy_norm))
print("Accuracy with Battacharrya classsifier: {0:.2f} %".format(accuracy_battacha))
#print("Accuracy with KL classsifier: {0:.2f} %".format(accuracy_KL))

Accuracy with euclidian classsifier: 82.50 %
Accuracy with normalized classsifier: 82.50 %
Accuracy with Battacharrya classsifier: 85.00 %
